In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM
from astropy.table import Table
import GCRCatalogs
from clmm import ShearAzimuthalAverager
from clmm import datatypes

### Read in the extragalactic catalog
- identify the most massive halo
- create a cluster dictionary (ra, dec, z)

In [2]:
extragalactic_cat = GCRCatalogs.load_catalog('proto-dc2_v2.1.2_test')

# get a massive halo!
massive_halos = extragalactic_cat.get_quantities(['halo_mass', 'redshift','ra', 'dec'],\
                                                 filters=['halo_mass > 1e14','is_central==True'])

m = massive_halos['halo_mass']
select = (m == np.max(m))
ra_cl = massive_halos['ra'][select][0]
dec_cl = massive_halos['dec'][select][0]
z_cl = massive_halos['redshift'][select][0]
m_cl = m[select][0]
print(m_cl, ra_cl, dec_cl, z_cl)

# make a dictionary for cluster
cl_dict = {'z':z_cl, 'ra':ra_cl, 'dec': dec_cl}


/pbs/throng/lsst/software/desc/python/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


ValueError: Catalog file /global/projecta/projectdirs/lsst/groups/CS/descqa/catalog/ANL_AlphaQ_v2.1.2.hdf5 does not exist

### Apply coordinates and redshift cuts to select backgroud galaxies around the cluster

In [3]:
# get galaxies around it
ra_min, ra_max = ra_cl-0.3, ra_cl+0.3
dec_min, dec_max = dec_cl-0.3, dec_cl+0.3
z_min = z_cl + 0.1
z_max = 1.5

coord_filters = [
    'ra >= {}'.format(ra_min),
    'ra < {}'.format(ra_max),
    'dec >= {}'.format(dec_min),
    'dec < {}'.format(dec_max),
]
z_filters = ['redshift >= {}'.format(z_min),'redshift < {}'.format(z_max)]

gal_cat = extragalactic_cat.get_quantities(['galaxy_id', 'ra', 'dec', 'shear_1', 'shear_2', 'shear_2_phosim', 'shear_2_treecorr',\
                                            'redshift', 'convergence'], filters=(coord_filters + z_filters))

# store the results into an astropy table
t = Table([gal_cat['galaxy_id'],gal_cat['ra'],gal_cat['dec'],gal_cat['shear_1'],\
          gal_cat['shear_2'],gal_cat['redshift'],gal_cat['convergence']], \
          names=('id','ra','dec', 'gamma1', 'gamma2', 'z', 'kappa'))

In [4]:
# Build the GalaxyCluster object
new_t = datatypes.GCData('source catalog', {'filters': coord_filters + z_filters}, t)

### Computes the tangential and cross shear profiles

#### Default binning

In [7]:
# create an object, given cluster dictionary and galaxy astropy table
saa = ShearAzimuthalAverager(cl_dict, new_t)

# compute tangential and cross shear for each galaxy
saa.compute_shear()

# make the binned profile
binned_profile = saa.make_shear_profile()

saa.plot_profile()   

TypeError: tuple indices must be integers or slices, not str

### Set up a model, using the Dallas group tools

In [ ]:
import sys
sys.path.append('../models/')
import clmm.models.CLMM_densityModels_beforeConvertFromPerH as dallas
import numpy as np
import colossus.cosmology.cosmology as Cosmology
import matplotlib.pyplot as plt
import time

zL = z_cl
chooseCosmology = 'WMAP7-ML' #Choose cosmology used
cosmo = Cosmology.setCosmology(chooseCosmology)

mdef = '200c'
M = m_cl*cosmo.h
c = 4.
r = np.linspace(0.5, 3., 100) #Mpc
r = r*cosmo.h #Mpc/h

testProf= dallas.nfwProfile(M = M, c = c, zL = zL, mdef = mdef, \
                        chooseCosmology = chooseCosmology, esp = None)


In [ ]:
z_infty = 300
zL_arr = np.zeros(len(t['z'])) + zL

In [ ]:
D_LS = (cosmo.comovingDistance(z_min=0.,z_max=t['z']) - cosmo.comovingDistance(z_min=0.,z_max=zL))/(1.+t['z'])
D_Linfty = (cosmo.comovingDistance(z_min=0., z_max=z_infty) - cosmo.comovingDistance(z_min=0., z_max=zL))/(1.+z_infty)
beta = D_LS * cosmo.angularDiameterDistance(z_infty)/cosmo.angularDiameterDistance(t['z'])/D_Linfty

In [ ]:
r_phys = binned_profile['radius']
gt_mod = np.zeros(len(r_phys))

for ir, r in enumerate(r_phys):
    gamtest_infty= testProf.deltaSigma(np.array([r])*0.7)/testProf.Sc(z_infty)
    kappa_infty =  testProf.Sigma(np.array([r])*0.7)/testProf.Sc(z_infty)
    gt_mod[ir] = np.mean(beta * gamtest_infty/(1-beta*kappa_infty))

In [ ]:
plt.plot(r_phys, gt_mod)
plt.errorbar(r_phys, binned_profile['g_t'], yerr=binned_profile['g_t_err'])
